# Site Data Monitoring

<div style="text-align: justify">

* Processing of site data for the Argus project for weekly meetings.
* Data should be in the form of at least 1 csv for GMR data, perhaps more to come in the form of temperature, pressure, density, flow, etc.

</div>

In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from datetime import datetime
from matplotlib.ticker import AutoMinorLocator, MultipleLocator

root = Path().absolute()
storage = Path('K://Josh/Argus_Project')

site_data = Path(f'{storage}/Site_Data/Sensor_Data')

<div style="text-align: justify">

* There are two files for the week beginning 11/06/2024, one that appears to be from the London plan (looking at the sensor data column headers) and one that was called "Libra" when it was sent to me, which is obvious.
* The csv files are oddly spaced, but I think pandas should be able to read it in quite easily.

</div>

In [54]:
london_data = pd.read_csv(
    filepath_or_buffer=Path(f'{site_data}/London_GMR_240611_240618.csv'),
    sep=',')
libra_data = pd.read_csv(
    filepath_or_buffer=Path(f'{site_data}/Libra_GMR_240611_240618.csv'),
    sep=',')

<div style="text-align: justify">

* Pandas should be able to read in the headers.
* Using the list(data_frame) function seems to be the recommended way of doing that.
* Try it:

</div>

In [55]:
london_headers = list(london_data)
libra_headers = list(libra_data)

<div style="text-align: justify">

* All of this data needs to be plotted as a function of the time stamp, so we should probably start by figuring out the time scale.
* Here, I think we can look to the temperature compensation notebook and use the same method, because in principle, it is the same data.

</div>

In [56]:
london_timestamp = london_data['timestamp']
libra_timestamp = libra_data['timestamp']

london_object = [
    datetime.strptime(date_string, "%d/%m/%Y %H:%M")
    for date_string in london_timestamp]
libra_object = [
    datetime.strptime(date_string, "%d/%m/%Y %H:%M")
    for date_string in libra_timestamp]

london_time = [obj.timestamp() for obj in london_object] 
libra_time = [obj.timestamp() for obj in libra_object]

london_corrected_time = [time - london_time[0] for time in london_time]
libra_corrected_time = [time - libra_time[0] for time in libra_time]

<div style="text-align: justify">

* That works with a few tweaks to the data input for the time/date stamps.
* Here the date is in the format dd/mm/YYYY HH:MM, not the same as the temperature compensation data.
* But a quick change to how it reads it in seems to work fine.
* Also I have corrected the time so that it starts at 0 and reads in seconds, could change this to minutes later I suppose if needed as it could get a bit ridiculous for the 7 days.
* Now let's plot them and see what they all look like.
* I tried plotting them all on the same figure and it doesn't look very nice because there are lines everywhere, there's actually quite a lot of data here.
* So let's try it a different way.
* Let's try plotting on a grid, much like my old SEM code does, where each header creates a new plot and we can look at it more easily.
* Also, yes, let's scale the time stamps back to minutes.

</div>

In [57]:
london_time_mins = [t / 60 for t in london_corrected_time]
london_time_hours = [t / 60 for t in london_time_mins]
london_number_columns = len(london_data.columns) - 1
london_number_rows = (london_number_columns + 1) // 2

fig, axes = plt.subplots(
    nrows=london_number_rows,
    ncols=2,
    figsize=[15 * 0.393701, london_number_rows * 9 * 0.393701],
    dpi=600)

if london_number_rows > 1:
    axes = axes.flatten()
else:
    axes = np.array([axes]).flatten()

for index, header in enumerate(london_headers[1: ]):
    axes[index].plot(
        london_time_hours,
        london_data[f'{header}'],
        f'C{index}',
        lw=1,
        label=f'{header}')
    axes[index].set_title(
        f'{header}',
        fontweight='bold',
        fontsize=6)
    axes[index].set_xlabel(
        'Time [hours]',
        fontweight='bold',
        fontsize=8)
    axes[index].set_ylabel(
        'Peak Position [px]',
        fontweight='bold',
        fontsize=8)
    axes[index].grid(
        True,
        alpha=0.5)
    axes[index].tick_params(
        axis='both',
        labelsize=6)
    axes[index].xaxis.set_major_locator(MultipleLocator(24))
    axes[index].xaxis.set_minor_locator(AutoMinorLocator())
    axes[index].yaxis.set_minor_locator(AutoMinorLocator())
fig.tight_layout()
plt.savefig(
    Path(f'{site_data}/London_GMR_240611_240618.png'),
    bbox_inches='tight')
plt.clf()


<Figure size 3543.31x8503.94 with 0 Axes>

In [58]:
libra_time_mins = [t / 60 for t in libra_corrected_time]
libra_time_hours = [t / 60 for t in libra_time_mins]
libra_number_columns = len(libra_data.columns) - 1
libra_number_rows = (libra_number_columns + 1) // 2

fig, axes = plt.subplots(
    nrows=libra_number_rows,
    ncols=2,
    figsize=[15 * 0.393701, libra_number_rows * 9 * 0.393701],
    dpi=600)

if libra_number_rows > 1:
    axes = axes.flatten()
else:
    axes = np.array([axes]).flatten()

for index, header in enumerate(libra_headers[1: ]):
    axes[index].plot(
        libra_time_hours,
        libra_data[f'{header}'],
        f'C{index}',
        lw=1,
        label=f'{header}')
    axes[index].set_title(
        f'{header}',
        fontweight='bold',
        fontsize=6)
    axes[index].set_xlabel(
        'Time [hours]',
        fontweight='bold',
        fontsize=8)
    axes[index].set_ylabel(
        'Peak Position [px]',
        fontweight='bold',
        fontsize=8)
    axes[index].grid(
        True,
        alpha=0.5)
    axes[index].tick_params(
        axis='both',
        labelsize=6)
    axes[index].xaxis.set_major_locator(MultipleLocator(24))
    axes[index].xaxis.set_minor_locator(AutoMinorLocator())
    axes[index].yaxis.set_minor_locator(AutoMinorLocator())
fig.tight_layout()
plt.savefig(
    Path(f'{site_data}/Libra_GMR_240611_240618.png'),
    bbox_inches='tight')
plt.clf()

<Figure size 3543.31x8503.94 with 0 Axes>